In [9]:
%pylab
!pip install celluloid
!pip install ffmpeg
!pip install termcolor
!pip install wget
!pip install scipy
!pip install numpy
!pip install matplotlib
!pip install sympy
import numpy as np
import matplotlib.pyplot as plt
import os.path as path
import scipy as sp
import wget
import sympy as sym
from matplotlib import rc 
from celluloid import Camera
from scipy import integrate
from termcolor import colored
from IPython.display import display, Math, Latex

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


/srv/conda/envs/notebook/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['rc', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [12]:
print(colored('Punto 2 ', attrs=['bold']))

#Definimos la función 

def funPunto2(t,n):
    f = 2 * (((-1)**(n-1))* np.sin(n*t ))/n
    return f

t = np.linspace(-np.pi,np.pi,100)
n = 1
nt = 51
Fourier = 0
fig = plt.figure(figsize=(5,5 ))
camera = Camera(fig)
plt.title('Primeros 50 modos')
plt.xlabel('Tiempo')


while n < nt:
    Fourier += funPunto2(t,n)
    plt.plot(t,Fourier,c='cadetblue')
    plt.legend([f'Modo {n}'])
    camera.snap()
    n += 1 
    
animation = camera.animate()
animation.save('modos.gif', writer='imagemagick')

Punto 2 


In [6]:
print(colored('Punto 3 ', attrs=['bold']))
display(Math(r' \int^{T/2}_{-T/2}|f(x)|^2 = \sum^{\infty}_{n=1} (a_n^2+b_n^2)'))

tiempo = np.linspace(0, 15, 1500)


def darCoef_a0( peri_, exp):
    t = sym.Symbol('t', Real = True)
    f = t**exp
    a0 = (2/peri_)*sym.integrate(f, (t, -peri_/2, peri_/2))
    return a0



def darCoef_an(peri_, exp):
    t = sym.Symbol('t', Real = True)
    n = sym.Symbol('n', Real  = True)
    f = t**exp
    fun = f*sym.cos(n*t)
    an = (2/peri_)*sym.integrate(fun, (t, -peri_/2, peri_/2))
    return an

def darCoef_bn(peri_, exp):
    t = sym.Symbol('t', Real = True)
    n = sym.Symbol('n', Real = True)
    f = t**exp
    fun = f*sym.sin(n*t)
    bn = (2/peri_)*sym.integrate(fun, (t, -peri_/2, peri_/2))
    return bn



#Rimman para x = 4
def parsevalt2(exp, peri_):
    t = sym.Symbol('t', Real = True)
    f = (t**exp)**2
    suma = (((1/peri_)*sym.integrate(f, (t, -peri_/2, peri_/2))) - (darCoef_a0(peri_, exp)**2))/8
    darCoef_bn(peri_, exp)
    return suma


def respuesta(exp, peri_):
    t = sym.Symbol('t', Real = True)
    f = ((1/12)*(t*(t**exp-np.pi**2)))**2
    suma = 1/np.pi * sym.integrate(f, (t, -peri_/2, peri_/2))
    return suma

a = respuesta(2, 2*np.pi)
print(f'De modo que la función zeta de rimman para x = 6 se tiene:{a}')


Punto 3 


<IPython.core.display.Math object>

De modo que la función zeta de rimman para x = 6 se tiene:1.01734306198445


In [7]:
print(colored('Punto 4 ', attrs=['bold']))
def fun(t):
    return np.exp(-0.1*t)*np.sin(t)

def df_a(t):
    return -0.1*np.exp(-0.1*t)*np.sin(t)+np.cos(t)*np.exp(-0.1*t)

def df_d(t):
    D = []
    for i in range(len(t)):
        a = (fun(t[i])-fun(t[i]-0.1))/0.1
        D.append(a)
    return D

Frec = 50 
paso= 4*np.pi/100 
f0 = 5
N = int (10*Frec/ f0) 

t = np.linspace(-2*np.pi,(N-1)*paso-2*np.pi,N)

f_step=Frec/N
freq=np.linspace(-25, (N-1)*f_step-25, int(N))

f=fun(t)
FFT=np.fft.fft(f)

freq=np.fft.fftshift(freq)
DFFT=freq*FFT*(1j)
d_esp=np.real(np.fft.ifft(DFFT))

plt.figure()
plt.plot(t,d_esp,c='r',label='Derivada espectral')
plt.plot(t,df_a(t),c='b',label='Derivada exacta')
plt.plot(t,df_d(t),c='y',label='Derivada derecha')
plt.legend()
plt.show()
plt.savefig('Derivada espectral.png')

Punto 4 


In [8]:
print(colored('Punto 5 ', attrs=['bold']))
file = 'ManchasSolares.txt'
data = np.loadtxt(file)
mask= data[:,0]>=1900
dat=data[mask]
año= dat[:,0]
mes= dat[:,1]
Nmanchas = dat[:,3]

Vmedio=Nmanchas-np.mean(Nmanchas)

X=np.fft.fft(Vmedio)
freq=np.fft.fftfreq(len(Vmedio),1)
mask1 = freq < 0
X[mask1] = 0

XMag_=np.abs(X)
XMag2_=np.argmax(XMag_)
freq_mes=freq[XMag2_]
freq_año=freq_mes*12
Periodo=1/freq_año

mask2=freq!=freq_mes
X[mask2]=0
Dominante=np.fft.ifft(2*X)
x=año+mes/12

plt.figure(figsize=(8,6))
plt.title(r'Periodo {:.2f} en años '.format(Periodo))
plt.plot(x,Nmanchas,label='Datos')
plt.plot(x,np.real(Dominante)+np.mean(Nmanchas),c='r',label='Frecuencia dominante')

plt.xlabel('Frecuencia [1/mes]')
plt.ylabel('Norma FFT')
plt.legend()
plt.savefig('PeriodoManchasS.png')

Punto 5 
